In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.io.json import json_normalize
import json
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/2011-data/"))
print(os.listdir("../input/mag-2011"))

# Any results you write to the current directory are saved as output.

['scopus-data', '2014-data', '2011-data', 'mag-2011']
['2011.json']
['mag_2011.csv']


In [58]:
# JSON_COLUMNS = ["AA","F","J"]
# df = pd.read_csv("../input/mag-2011/mag_2011.csv",converters={column: json.loads for column in JSON_COLUMNS})
df = pd.read_csv("../input/mag-2011/mag_2011.csv")

In [59]:
df.shape

(1177000, 16)

In [60]:
# Make more sensible column names
rename_dict = {"AA": "Authors", 
               "C": "Conference", 
               "CC": "Citation_count", 
               "D": "Date", 
               "F": "Domain", 
               "J": "Journal", 
               "RId": "References", 
               "Ti": "Title", 
               "VFN": "Venue", 
               "W": "Words", 
               "Y": "Year", 
               "logprob":"Popularity",
               "Unnamed: 0": "Row_num"}
df.rename(index=str, columns=rename_dict, inplace=True)

In [61]:
df.head()

,Row_num,Authors,Conference,Citation_count,Date,ECC,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity
0,0,"[{'S': 1, 'AuId': 2895256545, 'AfN': 'national...",NaN,38293,2011-04-01,38293,"[{'FId': 2777018310, 'FN': 'caltech 101'}, {'F...",2153635508,"{'JN': 'acm t intel syst tec', 'JId': 2492086750}","[2148603752, 2119821739, 2109943925, 217200036...",libsvm a library for support vector machines,ACM Transactions on Intelligent Systems and Te...,"['library', 'libsvm', 'machines', 'support', '...",2011,95072,-13.274
1,1,"[{'AuId': 581716687, 'S': 1, 'AuN': 'ahmedin j...",NaN,46871,2011-01-01,46871,"[{'FId': 2779372377, 'FN': 'lung cancer suscep...",2613543509,NaN,"[1981989535, 2095761465, 2132528779, 208930075...",global cancer statistics,Ca,"['cancer', 'global', 'statistics']",2011,109169,-13.460
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508
3,3,"[{'S': 1, 'AuId': 2121939561, 'AfN': 'universi...",NaN,42355,2011-07-06,42355,"[{'FId': 87146676, 'FN': 'gsp algorithm'}, {'F...",2140190241,NaN,"[2339804494, 2156909104, 2432517183, 210286575...",data mining concepts and techniques,NaN,"['concepts', 'data', 'mining', 'techniques']",2011,114591,-13.524
4,4,"[{'AuId': 2300357236, 'S': 1, 'AuN': 'laurence...",NaN,13533,2011-01-01,13533,NaN,2204447578,NaN,NaN,analise de conteudo,NaN,"['analise', 'conteudo']",2011,158426,-13.924


In [62]:
df.isna().sum()

Row_num                 0
Authors                 0
Conference        1115805
Citation_count          0
Date                    0
ECC                     0
Domain               6085
Id                      0
Journal            374096
References         186434
Title                   0
Venue              276961
Words                  12
Year                    0
kesEntityId             0
Popularity              0
dtype: int64

In [63]:
def preprocess(df):
    df = df.dropna(subset=['Venue'])
    return df

In [64]:
df = preprocess(df)

In [65]:
df.isna().sum()

Row_num                0
Authors                0
Conference        838853
Citation_count         0
Date                   0
ECC                    0
Domain              1558
Id                     0
Journal            97186
References         84460
Title                  0
Venue                  0
Words                  7
Year                   0
kesEntityId            0
Popularity             0
dtype: int64

In [66]:
df.head()

,Row_num,Authors,Conference,Citation_count,Date,ECC,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity
0,0,"[{'S': 1, 'AuId': 2895256545, 'AfN': 'national...",NaN,38293,2011-04-01,38293,"[{'FId': 2777018310, 'FN': 'caltech 101'}, {'F...",2153635508,"{'JN': 'acm t intel syst tec', 'JId': 2492086750}","[2148603752, 2119821739, 2109943925, 217200036...",libsvm a library for support vector machines,ACM Transactions on Intelligent Systems and Te...,"['library', 'libsvm', 'machines', 'support', '...",2011,95072,-13.274
1,1,"[{'AuId': 581716687, 'S': 1, 'AuN': 'ahmedin j...",NaN,46871,2011-01-01,46871,"[{'FId': 2779372377, 'FN': 'lung cancer suscep...",2613543509,NaN,"[1981989535, 2095761465, 2132528779, 208930075...",global cancer statistics,Ca,"['cancer', 'global', 'statistics']",2011,109169,-13.460
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508
5,5,"[{'AuId': 2047820406, 'S': 1, 'AuN': 'peter m ...",NaN,10603,2011-01-01,10603,"[{'FId': 128954960, 'FN': 'converged infrastru...",2275530856,NaN,NaN,the nist definition of cloud computing,Special Publication (NIST SP) - 800-145,"['cloud', 'computing', 'definition', 'nist']",2011,163742,-13.963
6,6,"[{'S': 1, 'AuId': 2120435251, 'AfN': 'isrec', ...",NaN,32471,2011-03-01,32471,"[{'FId': 2776199265, 'FN': 'the hallmarks of c...",2117692326,"{'JN': 'cell', 'JId': 110447773}","[2034269086, 2097259163, 2023427658, 196408120...",hallmarks of cancer the next generation,Cell,"['cancer', 'generation', 'hallmarks', 'next']",2011,193981,-14.161


In [67]:
df_pub = pd.read_csv("../input/scopus-data/ProcessedScopusData.csv")

In [68]:
df_pub.head()

,Unnamed: 0,Scopus SourceID,Title,CiteScore,Percentile,Citation Count,Scholarly Output,Percent Cited,SNIP,SJR,RANK,Rank Out Of,Publisher,Type,OpenAccess,Scopus ASJC Code (Sub-subject Area),Scopus Sub-Subject Area,Quartile,Top 10% (CiteScore Percentile),Scopus SourceID.1,Print-ISSN,E-ISSN,Subject Area
0,0,28773.0,Ca-A Cancer Journal for Clinicians,130.47,99.0,16961.0,130.0,70.0,88.164,61.786,1.0,120.0,Wiley-Blackwell,Journal,NO,2720.0,Hematology,Quartile 1,Top 10%,https://www.scopus.com/sourceid/28773,00079235,15424863,Medicine
1,1,28773.0,Ca-A Cancer Journal for Clinicians,130.47,99.0,16961.0,130.0,70.0,88.164,61.786,1.0,323.0,Wiley-Blackwell,Journal,NO,2730.0,Oncology,Quartile 1,Top 10%,https://www.scopus.com/sourceid/28773,00079235,15424863,Medicine
2,2,19434.0,MMWR. Recommendations and reports : Morbidity ...,63.12,99.0,1010.0,16.0,100.0,32.534,34.638,1.0,87.0,Centers for Disease Control and Prevention (CDC),Journal,YES,2713.0,Epidemiology,Quartile 1,Top 10%,https://www.scopus.com/sourceid/19434,10575987,15458601,Medicine
3,3,19434.0,MMWR. Recommendations and reports : Morbidity ...,63.12,99.0,1010.0,16.0,100.0,32.534,34.638,1.0,241.0,Centers for Disease Control and Prevention (CDC),Journal,YES,3306.0,Health(social science),Quartile 1,Top 10%,https://www.scopus.com/sourceid/19434,10575987,15458601,Social Sciences
4,4,19434.0,MMWR. Recommendations and reports : Morbidity ...,63.12,99.0,1010.0,16.0,100.0,32.534,34.638,1.0,106.0,Centers for Disease Control and Prevention (CDC),Journal,YES,2307.0,"Health, Toxicology and Mutagenesis",Quartile 1,Top 10%,https://www.scopus.com/sourceid/19434,10575987,15458601,Environmental Science


In [70]:
def append_pub_score(df):
    print("Unique Publications from Scopus:",len(df_pub.Title.unique()))
    # Analysing Common Publications
    vfn_list = df.Venue.unique().tolist()
    scopus_list = df_pub.Title.unique().tolist()
    count = 0
    for pub in scopus_list:
        if pub in vfn_list:
            count +=1
    print("Publications present in Dataset:",count)
    print("Shape Before",df.shape)
    df = df[df.Venue.isin(scopus_list)]
    print("Shape After",df.shape)
    #score appending
    score_dict = pd.Series(df_pub.SJR.values,index=df_pub.Title).to_dict()
    df["Publication_Rank"] = df.apply(lambda row: score_dict[row["Venue"]],axis = 1)
    return df

def extract_field(row):
    val = row["Domain"]
    index = val.rfind("FN")
    val = val[index+3:len(val)]
    val = val.strip(":}] '")
    return val

def get_num_authors(row):
    vals = row["Authors"].split("},")
    return len(vals)

In [71]:
df = append_pub_score(df)

Unique Publications from Scopus: 1231
Publications present in Dataset: 896
Shape Before (900039, 16)
Shape After (123825, 16)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [79]:
label_dict = {"C":0,"J":1,"CJ":2,"O":3}
def find_conference_type(row):
    if pd.isnull(row["Conference"]) and pd.isnull(row["Journal"]):
        return label_dict["O"]
    elif pd.isnull(row["Conference"]):
        return label_dict["J"]
    elif pd.isnull(row["Journal"]):
        return label_dict["C"]
    else:
        return label_dict["CJ"]

In [80]:
df["Publication Type"] = df.apply(lambda row:find_conference_type(row),axis=1)

In [81]:
df["Number_Of_authors"] = df.apply(lambda row:get_num_authors(row), axis = 1)

In [ ]:
# df["Subject Field"] = df.apply(lambda row:extract_field(row),axis=1)

In [82]:
df.head()

,Row_num,Authors,Conference,Citation_count,Date,ECC,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity,Publication_Rank,Publication Type,Number_Of_authors
0,0,"[{'S': 1, 'AuId': 2895256545, 'AfN': 'national...",NaN,38293,2011-04-01,38293,"[{'FId': 2777018310, 'FN': 'caltech 101'}, {'F...",2153635508,"{'JN': 'acm t intel syst tec', 'JId': 2492086750}","[2148603752, 2119821739, 2109943925, 217200036...",libsvm a library for support vector machines,ACM Transactions on Intelligent Systems and Te...,"['library', 'libsvm', 'machines', 'support', '...",2011,95072,-13.274,0.874,1,2
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508,5.475,1,7
6,6,"[{'S': 1, 'AuId': 2120435251, 'AfN': 'isrec', ...",NaN,32471,2011-03-01,32471,"[{'FId': 2776199265, 'FN': 'the hallmarks of c...",2117692326,"{'JN': 'cell', 'JId': 110447773}","[2034269086, 2097259163, 2023427658, 196408120...",hallmarks of cancer the next generation,Cell,"['cancer', 'generation', 'hallmarks', 'next']",2011,193981,-14.161,25.137,1,3
7,7,"[{'S': 1, 'AuId': 317192602, 'AfN': 'french in...",NaN,12801,2011-02-01,12801,"[{'FId': 24138899, 'FN': 'instance based learn...",2101234009,"{'JN': 'jmlr', 'JId': 118988714}","[2153635508, 2118585731, 2063978378, 209736028...",scikit learn machine learning in python,Journal of Machine Learning Research,"['learn', 'learning', 'machine', 'python', 'sc...",2011,264962,-14.513,1.271,1,16
28,28,"[{'S': 1, 'AuId': 281745206, 'AfN': 'universit...",NaN,7383,2011-03-01,7383,"[{'FId': 2776282453, 'FN': 'milan criteria'}, ...",2163403599,"{'JN': 'hepatology', 'JId': 156174673}","[1971837077, 1994193851, 2057378223, 213032537...",management of hepatocellular carcinoma an update,Hepatology,"['carcinoma', 'hepatocellular', 'management', ...",2011,489309,-15.158,5.541,1,2


In [84]:
df.shape

(123825, 19)

In [87]:
def tidy_split(df, column, sep='|', keep=False):
    indexes = list()
    id_values = list()
    name_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            id_values.append(presplit)
        for value in values:
            indexes.append(i)
            val = value
            index1 = val.find("AuId")
            index2 = val.find("AfN")
            index3 = val.find("'S'")
            if index3 < index1:
                val = val[index1+4:index2]
                val = val.strip("', :")
            else:
                val = val[index1+4:index3]
                val = val.strip("', :")                
            id_values.append(val)
            index1 = value.find("AuN")
            val = value[index1+3:]
            val = val.strip("': }]")
            name_values.append(val)
    new_df = df.iloc[indexes, :].copy()
    new_df[column+"_Id"] = id_values
    new_new_df = new_df.iloc[:, :].copy()
    new_new_df[column+"_Name"] = name_values
    return new_new_df

In [88]:
df_big = tidy_split(df,"Authors","},")

In [89]:
df_big.head(10)

,Row_num,Authors,Conference,Citation_count,Date,ECC,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity,Publication_Rank,Publication Type,Number_Of_authors,Authors_Id,Authors_Name
0,0,"[{'S': 1, 'AuId': 2895256545, 'AfN': 'national...",NaN,38293,2011-04-01,38293,"[{'FId': 2777018310, 'FN': 'caltech 101'}, {'F...",2153635508,"{'JN': 'acm t intel syst tec', 'JId': 2492086750}","[2148603752, 2119821739, 2109943925, 217200036...",libsvm a library for support vector machines,ACM Transactions on Intelligent Systems and Te...,"['library', 'libsvm', 'machines', 'support', '...",2011,95072,-13.274,0.874,1,2,2895256545,chihchung chang
0,0,"[{'S': 1, 'AuId': 2895256545, 'AfN': 'national...",NaN,38293,2011-04-01,38293,"[{'FId': 2777018310, 'FN': 'caltech 101'}, {'F...",2153635508,"{'JN': 'acm t intel syst tec', 'JId': 2492086750}","[2148603752, 2119821739, 2109943925, 217200036...",libsvm a library for support vector machines,ACM Transactions on Intelligent Systems and Te...,"['library', 'libsvm', 'machines', 'support', '...",2011,95072,-13.274,0.874,1,2,2786671625,chihjen lin
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508,5.475,1,7,2164450181,koichiro tamura
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508,5.475,1,7,2164450181,koichiro tamura
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508,5.475,1,7,2609879285,daniel peterson
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508,5.475,1,7,2705958286,nicholas peterson
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508,5.475,1,7,2307130762,glen stecher
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508,5.475,1,7,2302581886,masatoshi nei
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biolo

In [56]:
df_big.shape

(4714564, 19)